In [4]:
%pip install psycopg2-binary
%pip install pandas
%pip install numpy
%pip install scipy
%pip install ipywidgets
%pip install requests
%pip install boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Metadata Wide Association Study (MWAS)
## rfamily run V2

- statistical method to identify associations between BioSample metadata and provided count data
- inspired by GWAS (Genome-Wide Association Study) 

In [2]:
# import required libraries
import psycopg2
import pandas as pd
import numpy as np
import scipy.stats as stats
import requests
import warnings
import logging
import sys
import io
import pickle
import csv
import boto3

In [115]:
# --------------- HELPER FUNCTIONS ---------------
# - specific to the rfamily run

def family_biosamples_mapper(row):
    if row['bio_sample'] in family_biosamples_n_reads:
        family_biosamples_n_reads[row['bio_sample']].append(row['n_reads'])
        family_biosample_spots[row['bio_sample']].append(row['spots'])
        # family_biosample_runs[row['bio_sample']].append(row['run_id']) # TO BE USED IF MAPPING IS DONE FROM RUNS
    else:
        family_biosamples_n_reads[row['bio_sample']] = [row['n_reads']]
        family_biosample_spots[row['bio_sample']] = [row['spots']]
        # family_biosample_runs[row['bio_sample']] = [row['run_id']] # TO BE USED IF MAPPING IS DONE FROM RUNS

    
def family_bioproject_mapper(row):
    try:
        return srarun_bioprojects_map[row['bio_sample'][1:-1]]
    except:
        return np.nan

def get_n_reads(row):
    # try to get the n_reads from family_biosamples_n_reads, then from srarun_biosamples_map, then default to np.nan
    #   - tries to get exact n_reads from serratus
    #   - if run in serratus but no n_reads, then set to 0 (from srarun_biosamples_map)
    #   - if not run in serratus, then set to np.nan
    try:
        return family_biosamples_n_reads[row['biosample_id'][1:-1]]
    except:
        try:
            return srarun_biosamples_map[row['biosample_id'][1:-1]]
        except:
            return np.nan

def get_n_spots(row):
    # try to get the n_spots from family_biosample_spots, otherwise default to 0 
    ##### CHECK THIS ##### will have divide by 0 error if the mapping is wrong
    try:
        return family_biosample_spots[row['biosample_id'][1:-1]]
    except:
        return 0

def get_log_fold_change(true, false):
    # calculate the log fold change of true with respect to false
    #   - if true and false is 0, then return 0

    if true == 0 and false == 0:
        return 0
    elif true == 0:
        return -np.inf
    elif false == 0:
        return np.inf
    else:
        return np.log2(true / false)

# ---------------------------------------------

In [4]:
# load information from pickles
# ADJUST FILE PATH ON FINAL RUN
with open(f'pickles/rfam_df.pickle', 'rb') as f:
    rfam_df = pickle.load(f)
with open(f'pickles/srarun_df.pickle', 'rb') as f:
    srarun_df = pickle.load(f)

with open(f'pickles/srarun_biosamples_map.pickle', 'rb') as handle:
    # mapping from biosamples (in srarun_df) to 0 for default count
    srarun_biosamples_map = pickle.load(handle) 
with open(f'pickles/srarun_bioprojects_map.pickle', 'rb') as handle:
    # mapping from biosamples (in srarun_df) to bioprojects
    srarun_bioprojects_map = pickle.load(handle)

In [5]:
# set up variables for API call
base_url = 'https://serratus-biosamples.s3.us-east-1.amazonaws.com/bioprojects_csv/'
headers = {'Host': 'serratus-biosamples.s3.amazonaws.com'}

s3 = boto3.resource('s3')
bucket = s3.Bucket('serratus-biosamples')

In [99]:
family = 'Marnaviridae-2'

# subset the rfam_df to the family of interest
family_df = rfam_df[rfam_df['family_group'] == family]
family_df = pd.merge(family_df[['run_id', 'n_reads']], srarun_df[['run', 'bio_sample', 'spots']], left_on='run_id', right_on='run')
family_df


,run_id,n_reads,run,bio_sample,spots
0,SRR12422552,1,SRR12422552,SAMN15401089,25910571
1,SRR12166615,1,SRR12166615,SAMN15465386,44937715
2,SRR12358964,7,SRR12358964,SAMN15688124,22947931
3,SRR12358965,8,SRR12358965,SAMN15688123,21032993
4,SRR1232069,3,SRR1232069,SAMN02679987,78065807
...,...,...,...,...,...
231825,SRR9994063,6,SRR9994063,SAMN12598143,36968628
231826,SRR9994051,3,SRR9994051,SAMN12598115,39022407
231827,SRR9994060,5,SRR9994060,SAMN12598146,34924479
231828,SRR9994038,4,SRR9994038,SAMN12598128,40115142


In [100]:

# map the biosamples in the family to n_reads and spots
family_biosamples_n_reads = {}
family_biosample_spots = {}
family_df.apply(family_biosamples_mapper, axis=1)

# average the n_reads and spots for each biosample
for biosample in family_biosamples_n_reads:
    family_biosamples_n_reads[biosample] = np.mean(family_biosamples_n_reads[biosample])
    family_biosample_spots[biosample] = np.mean(family_biosample_spots[biosample])

# map the biosamples in the family to their bioprojects
family_df['bio_project'] = family_df.apply(family_bioproject_mapper, axis=1)

# get the bioprojects for the family and remove None from the list
bioprojects = list(family_df['bio_project'].unique())
if None in bioprojects:
    bioprojects.remove(None)

In [108]:
family_df[family_df['bio_project']=='PRJNA244607']

,run_id,n_reads,run,bio_sample,spots,bio_project
874,SRR1266984,3,SRR1266984,SAMN02727171,35164879,PRJNA244607
896,SRR1266985,10,SRR1266985,SAMN02727172,18826677,PRJNA244607
967,SRR1266998,6,SRR1266998,SAMN02727185,19966548,PRJNA244607
975,SRR1267006,7,SRR1267006,SAMN02727193,19212772,PRJNA244607
986,SRR1267019,3,SRR1267019,SAMN02727206,15423599,PRJNA244607
1001,SRR1267025,2,SRR1267025,SAMN02727212,12689498,PRJNA244607
1005,SRR1267032,8,SRR1267032,SAMN02727219,21283075,PRJNA244607
1044,SRR1267040,5,SRR1267040,SAMN02727227,20289308,PRJNA244607
72297,SRR1266990,9,SRR1266990,SAMN02727177,25649196,PRJNA244607
72298,SRR1266993,5,SRR1266993,SAMN02727180,21539436,PRJNA244607


In [101]:
len(bioprojects)
'PRJNA244607' in bioprojects

True

In [132]:

def statistic(x, y, axis):
    # if -ve, then y is larger than x
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

bioprojects = ['PRJNA244607']
for bioproject_id in bioprojects:
    print(f'Processing {bioproject_id}')
    # get the csv file for the bioproject
    url = base_url + bioproject_id + '.csv'
    
    # connection reset by peer error?
    try:
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            # skip the bioproject if it doesn't exist
            print(f'Error: {response.status_code} for {bioproject_id}')
            continue
    except Exception as e:
        print(f'Error occurred while making API call: {e}')
        continue

     # read the csv file into a dataframe
    bp_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), quoting=csv.QUOTE_NONE)

    # skip if there are two rows or less
    num_rows = bp_df.shape[0]
    if num_rows <= 2:
        logger.warning(f'Skipping {bioproject_id}: {num_rows} rows')
        continue

    # add the n_reads and n_spots columns to the dataframe
    bp_df['n_reads'] = bp_df.apply(get_n_reads, axis=1)
    bp_df['n_spots'] = bp_df.apply(get_n_spots, axis=1)

    # add the rpm column to the dataframe
    bp_df['rpm'] = bp_df.apply(lambda row: row['n_reads'] / row['n_spots'] * 1000000 if row['n_spots'] != 0 else 0, axis=1)
    
    display(bp_df)
    # --------------- SELECTING COLUMNS TO TEST ---------------
    # get the list of testable columns in the dataframe
    remove_cols = {'biosample_id','n_reads','n_spots','rpm'}
    target_cols = list(set(bp_df.columns) - remove_cols)

    metadata_counts = {}

    # store the value counts if the column can be tested
    for col in target_cols:
        counts = bp_df[col].value_counts()
        n = len(counts)
        # skip if there is only one unique value or if all values are unique
        if n == 1:
            continue
        if n == num_rows:
            continue
        metadata_counts[col] = counts

    existing_samples = list()
    target_col_runs = {}

    # iterate through the values for all columns that can be tested
    for target_col, value_counts in metadata_counts.items():
        for target_term, count in value_counts.items():
            # skip if there is only one value
            if count == 1:
                continue

            # get the biosamples that correspond to the target term
            target_term_biosamples = list(bp_df[bp_df[target_col] == target_term]['biosample_id'])

            # check if the same biosamples are already stored and aggregate the columns
            target_col_name = f'{target_col}\t{target_term}'

            if target_term_biosamples in existing_samples:
                existing_key = list(target_col_runs.keys())[list(target_col_runs.values()).index(target_term_biosamples)]
                target_col_name = f'{existing_key}\r{target_col_name}'
                # update the dictionary with the new name
                target_col_runs[target_col_name] = target_col_runs.pop(existing_key)
            else:
                existing_samples.append(target_term_biosamples)
                target_col_runs[target_col_name] = target_term_biosamples

    print(target_col_runs)
    # --------------- RUN TTESTS ---------------
    # set up the columns for the output df
    output_cols = ['bioproject_id', 'family', 'metadata_field', 'metadata_value', 'num_true', 'num_false', 'mean_rpm_true', 'mean_rpm_false', 'sd_rpm_true', 'sd_rpm_false', 'fold_change', 'test_statistic', 'p_value']
    output_dict = {col: [] for col in output_cols}

    for target_col, biosamples in target_col_runs.items():
        num_true = len(biosamples)
        num_false = num_rows - num_true

        # regular t test code
        # get the rpm values for the target and remainng biosamples
        true_rpm = bp_df[bp_df['biosample_id'].isin(biosamples)]['rpm']
        false_rpm = bp_df[~bp_df['biosample_id'].isin(biosamples)]['rpm']
        
        # calculate desecriptive stats
        # NON CORRECTED VALUES
        mean_rpm_true = np.nanmean(true_rpm)
        mean_rpm_false = np.nanmean(false_rpm)
        sd_rpm_true = np.nanstd(true_rpm)
        sd_rpm_false = np.nanstd(false_rpm)

        # calculate fold change and check if any values are nan
        fold_change = get_log_fold_change(mean_rpm_true, mean_rpm_false)

        # use scipy permutation test
        res = stats.permutation_test((true_rpm, false_rpm), statistic=statistic, n_resamples=10000)
        p_value = res.pvalue
        test_statistic = res.statistic

        # use scipy ttest
        # t_statistic, p_value = stats.ttest_ind_from_stats(mean1=mean_rpm_true, std1=sd_rpm_true, nobs1=num_true, mean2=mean_rpm_false, std2=sd_rpm_false, nobs2=num_false, equal_var=False)

        # extract metadata_field (column names) and metadata_value (column values)
        metadata_tmp = target_col.split('\r') # pairs of metadata_field and metadata_value for aggregated columns
        metadata_field = '\t'.join(pair.split('\t')[0] for pair in metadata_tmp)
        metadata_value = '\t'.join(pair.split('\t')[1] for pair in metadata_tmp)

        # add values to output dict
        output_dict['bioproject_id'].append(bioproject_id)
        output_dict['family'].append(family)
        output_dict['metadata_field'].append(metadata_field)
        output_dict['metadata_value'].append(metadata_value)
        output_dict['num_true'].append(num_true)
        output_dict['num_false'].append(num_false)
        output_dict['mean_rpm_true'].append(mean_rpm_true)
        output_dict['mean_rpm_false'].append(mean_rpm_false)
        output_dict['sd_rpm_true'].append(sd_rpm_true)
        output_dict['sd_rpm_false'].append(sd_rpm_false)
        output_dict['fold_change'].append(fold_change)
        output_dict['test_statistic'].append(test_statistic)
        output_dict['p_value'].append(p_value)



    # create the output df and sort by p_value
    mwas_df = pd.DataFrame(output_dict)
    mwas_df = mwas_df.sort_values(by='p_value')

    
    # log if there are significant results
    # num_significant = mwas_df[mwas_df['p_value'] < 0.05].shape[0]
    # if num_significant > 0:
    #     logger.info(f'{bioproject_id} - {num_significant} significant results')
    
    # # write the mwas_df to a csv file on S3
    # key = f's3://serratus-biosamples/mwas/{family}/{bioproject_id}.csv'
    # mwas_df.to_csv(key, index=False)
    # logger.info(f'Finished {bioproject_id}')

    

Processing PRJNA244607


,cultivar,specimen_voucher,title,Sample name_db,BioSample_db,package,status,tissue,Name,last_update,...,when,paragraph,organism_taxonomy_name,biomaterial_provider,publication_date,biosample_id,age,n_reads,n_spots,rpm
0,"""nan""","""5093""","""Plant sample from Howea belmoreana""","""5093_floral""","""SAMN02727202""","""Plant.1.0""","""live""","""floral""","""Imperial College London""","""2015-04-16T06:54:56.107""",...,"""2015-04-16T06:54:56.107""","""nan""","""Howea belmoreana""","""Field Collection""","""2015-04-16T06:54:56.107""","""SAMN02727202""","""missing""",7.0,16730886.0,0.418388
1,"""nan""","""2388""","""Plant sample from Howea forsteriana""","""2388_leaf""","""SAMN02727213""","""Plant.1.0""","""live""","""leaf""","""Imperial College London""","""2015-04-16T06:54:56.737""",...,"""2015-04-16T06:54:56.737""","""nan""","""Howea forsteriana""","""Field Collection""","""2015-04-16T06:54:56.737""","""SAMN02727213""","""missing""",0.0,0.0,0.000000
2,"""nan""","""2637""","""Plant sample from Howea belmoreana""","""2637_leaf""","""SAMN02727186""","""Plant.1.0""","""live""","""leaf""","""Imperial College London""","""2015-04-16T06:54:57.367""",...,"""2015-04-16T06:54:57.367""","""nan""","""Howea belmoreana""","""Field Collection""","""2015-04-16T06:54:57.367""","""SAMN02727186""","""missing""",0.0,0.0,0.000000
3,"""nan""","""2386""","""Plant sample from Howea forsteriana""","""2386_root""","""SAMN02727211""","""Plant.1.0""","""live""","""root""","""Imperial College London""","""2015-04-16T06:54:56.613""",...,"""2015-04-16T06:54:56.613""","""nan""","""Howea forsteriana""","""Field Collection""","""2015-04-16T06:54:56.613""","""SAMN02727211""","""missing""",2.0,31620993.0,0.063249
4,"""nan""","""2564""","""Plant sample from Howea forsteriana""","""2564_floral""","""SAMN02727219""","""Plant.1.0""","""live""","""floral""","""Imperial College London""","""2015-04-16T06:54:57.060""",...,"""2015-04-16T06:54:57.060""","""nan""","""Howea forsteriana""","""Field Collection""","""2015-04-16T06:54:57.060""","""SAMN02727219""","""missing""",8.0,21283075.0,0.375886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,"""nan""","""4725""","""Plant sample from Howea belmoreana""","""4725_floral""","""SAMN02727197""","""Plant.1.0""","""live""","""floral""","""Imperial College London""","""2015-04-16T06:54:55.860""",...,"""2015-04-16T06:54:55.860""","""nan""","""Howea belmoreana""","""Field Collection""","""2015-04-16T06:54:55.860""","""SAMN02727197""","""missing""",2.0,15059910.0,0.132803
75,"""nan""","""5104""","""Plant sample from Howea forsteriana""","""5104_leaf""","""SAMN02727246""","""Plant.1.0""","""live""","""leaf""","""Imperial College London""","""2015-04-16T06:55:02.190""",...,"""2015-04-16T06:55:02.190""","""nan""","""Howea forsteriana""","""Field Collection""","""2015-04-16T06:55:02.190""","""SAMN02727246""","""missing""",0.0,0.0,0.000000
76,"""nan""","""4704""","""Plant sample from Howea belmoreana""","""4704_root""","""SAMN02727192""","""Plant.1.0""","""live""","""root""","""Imperial College London""","""2015-04-16T06:54:55.540""",...,"""2015-04-16T06:54:55.540""","""nan""","""Howea belmoreana""","""Field Collection""","""2015-04-16T06:54:55.540""","""SAMN02727192""","""missing""",0.0,0.0,0.000000
77,"""nan""","""2566""","""Plant sample from Howea forsteriana""","""2566_leaf""","""SAMN02727222""","""Plant.1.0""","""live""","""leaf""","""Imperial College London""","""2015-04-16T06:54:59.260""",...,"""2015-04-16T06:54:59.260""","""nan""","""Howea forsteriana""","""Field Collection""","""2015-04-16T06:54:59.260""","""SAMN02727222""","""missing""",0.0,0.0,0.000000


{'tissue\t"floral"': ['"SAMN02727202"', '"SAMN02727219"', '"SAMN02727209"', '"SAMN02727193"', '"SAMN02727195"', '"SAMN02727229"', '"SAMN02727225"', '"SAMN02727172"', '"SAMN02727177"', '"SAMN02727206"', '"SAMN02727232"', '"SAMN02727169"', '"SAMN02727239"', '"SAMN02727187"', '"SAMN02727235"', '"SAMN02727227"', '"SAMN02727221"', '"SAMN02727185"', '"SAMN02727237"', '"SAMN02727180"', '"SAMN02727175"', '"SAMN02727203"', '"SAMN02727212"', '"SAMN02727199"', '"SAMN02727245"', '"SAMN02727190"', '"SAMN02727179"', '"SAMN02727215"', '"SAMN02727242"', '"SAMN02727197"'], 'tissue\t"leaf"': ['"SAMN02727213"', '"SAMN02727186"', '"SAMN02727238"', '"SAMN02727216"', '"SAMN02727226"', '"SAMN02727230"', '"SAMN02727207"', '"SAMN02727174"', '"SAMN02727228"', '"SAMN02727181"', '"SAMN02727194"', '"SAMN02727210"', '"SAMN02727233"', '"SAMN02727200"', '"SAMN02727173"', '"SAMN02727170"', '"SAMN02727236"', '"SAMN02727240"', '"SAMN02727188"', '"SAMN02727176"', '"SAMN02727196"', '"SAMN02727205"', '"SAMN02727178"', '"SA

In [130]:
mwas_df

,bioproject_id,family,metadata_field,metadata_value,num_true,num_false,mean_rpm_true,mean_rpm_false,sd_rpm_true,sd_rpm_false,fold_change,test_statistic,p_value
0,PRJNA244607,Marnaviridae-2,tissue,"""floral""",30,49,0.267618,0.005650,0.166441,0.017907,5.565697,0.261967,0.000133
1,PRJNA244607,Marnaviridae-2,tissue,"""leaf""",30,49,0.000000,0.169498,0.000000,0.180090,-inf,-0.169498,0.000133
2,PRJNA244607,Marnaviridae-2,tissue,"""root""",19,60,0.014572,0.133809,0.026400,0.178202,-3.198915,-0.119237,0.001067
32,PRJNA244607,Marnaviridae-2,title\torganism_taxonomy_id\torganism_taxonomy...,"""Plant sample from Howea belmoreana""\t""145691""...",40,39,0.134804,0.074698,0.188739,0.126849,0.851713,0.060106,0.101327
33,PRJNA244607,Marnaviridae-2,title\torganism_taxonomy_id\torganism_taxonomy...,"""Plant sample from Howea forsteriana""\t""292773...",39,40,0.074698,0.134804,0.126849,0.188739,-0.851713,-0.060106,0.111459
29,PRJNA244607,Marnaviridae-2,specimen_voucher,"""4716""",2,77,0.324617,0.099431,0.324617,0.153495,1.706977,0.225187,0.122038
31,PRJNA244607,Marnaviridae-2,specimen_voucher,"""2378""",2,77,0.265581,0.100964,0.265581,0.158314,1.395307,0.164616,0.240182
24,PRJNA244607,Marnaviridae-2,specimen_voucher,"""2564""",2,77,0.215230,0.102272,0.160655,0.163058,1.073471,0.112958,0.354430
14,PRJNA244607,Marnaviridae-2,specimen_voucher,"""4704""",3,76,0.182826,0.102065,0.258556,0.158297,0.840990,0.080762,0.406773
15,PRJNA244607,Marnaviridae-2,specimen_voucher,"""4691""",3,76,0.000000,0.109282,0.000000,0.165804,-inf,-0.109282,0.418639


In [98]:
stats.permutation_test((a,b), statistic, permutation_type='independent', n_resamples=10000)

PermutationTestResult(statistic=-18.7, pvalue=0.00019998000199980003, null_distribution=array([-1.57384615,  1.29230769,  0.62      , ...,  1.82307692,
        1.43384615, -6.10307692]))